In [1]:
from ple.games.flappybird import FlappyBird
from ple import PLE
import numpy as np

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, Dropout
from keras import initializers, optimizers

from PIL import Image
np.set_printoptions(precision=3)
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)
from skimage.color import rgb2gray
from skimage.transform import resize

from collections import deque

couldn't import doomish
Couldn't import doom


Using TensorFlow backend.


In [2]:
dqn = load_model('temp/flappy-bucket-2/nickel600000.h5')

In [9]:
try:
    confirmation = int(input("confirm network reinitialisation ?")) 
except:
    print("failed")
    confirmation = 0
if(confirmation == 1):
    dqn = Sequential()
    #1st layer
    dqn.add(Conv2D(filters=16, kernel_size=(8,8), strides=4, activation="relu", input_shape=(67,104,4)))
    #2nd layer
    dqn.add(Conv2D(filters=32, kernel_size=(4,4), strides=2, activation="relu"))
    dqn.add(Flatten())
    #3rd layer
    dqn.add(Dense(units=256, activation="relu"))
    #output layer
    dqn.add(Dense(units=2, activation="linear"))
    
    my_adam = optimizers.Adam(lr=0.00001)
    dqn.compile(optimizer=my_adam, loss="mean_squared_error")
    #dqn.compile(optimizer="rmsprop", loss="mean_squared_error")
    print("network re-initialized")

confirm network reinitialisation ?1
network re-initialized


In [3]:
#----------------------------functions--------------------------


def BasicPolicy(state):
    if(state["player_y"] > (state["next_pipe_bottom_y"] + state["next_pipe_top_y"])/2):
        return(1)
    else:
        return(0)
    
def epsilon(step):
    final = 0.1
    fraction = 1/3
    if step < total_steps*fraction:
        return 1-(step)*(1-final)/(total_steps*fraction)
    return final

def clip_reward_2(r, game_over):
    if game_over:
        return(-1)
    if r ==1:
        return(1)
    return(0.1)
    #if ((state["player_y"] - (state["next_pipe_bottom_y"] + state["next_pipe_top_y"])/2) !=0):
    #    return(steps/500/np.sqrt(np.absolute(state["player_y"] - (state["next_pipe_bottom_y"] + state["next_pipe_top_y"])/2)))
    
    

def greedy_action(network, x):
    Q = network.predict(np.array([x]))
    return np.argmax(Q)

def process_screen(x):
    h = 130
    w = 80
    return 256*resize(rgb2gray(x), (w,h))[int(w/6):,0:int(4/5*h)]

def int_to_action(n):
    if n == 0:
        return(None)
    if n == 1:
        return(119)


In [4]:
#------------------------------Memory Buffer-----------------------------------
class MemoryBuffer:
    "An experience replay buffer using numpy arrays"
    def __init__(self, length, screen_shape, action_shape):
        self.length = length
        self.screen_shape = screen_shape
        self.action_shape = action_shape
        shape = (length,) + screen_shape
        self.screens_x = np.zeros(shape, dtype=np.uint8) # starting states
        self.screens_y = np.zeros(shape, dtype=np.uint8) # resulting states
        shape = (length,) + action_shape
        self.actions = np.zeros(shape, dtype=np.uint8) # actions
        self.rewards = np.zeros((length,1), dtype=np.float64) # rewards
        self.terminals = np.zeros((length,1), dtype=np.bool) # true if resulting state is terminal
        self.terminals[-1] = True
        self.index = 0 # points one position past the last inserted element
        self.size = 0 # current size of the buffer
    
    def append(self, screenx, a, r, screeny, d):
        self.screens_x[self.index] = screenx
        #plt.imshow(screenx)
        #plt.show()
        #plt.imshow(self.screens_x[self.index])
        #plt.show()
        self.actions[self.index] = a
        self.rewards[self.index] = r
        self.screens_y[self.index] = screeny
        self.terminals[self.index] = d
        self.index = (self.index+1) % self.length
        self.size = np.min([self.size+1,self.length])
    
    def stacked_frames_x(self, index):
        im_deque = deque(maxlen=4)
        pos = index % self.length
        for i in range(4): # todo
            im = self.screens_x[pos]
            im_deque.appendleft(im)
            test_pos = (pos-1) % self.length
            if self.terminals[test_pos] == False:
                pos = test_pos
        return np.stack(im_deque, axis=-1)
    
    def stacked_frames_y(self, index):
        im_deque = deque(maxlen=4)
        pos = index % self.length
        for i in range(4): # todo
            im = self.screens_y[pos]
            im_deque.appendleft(im)
            test_pos = (pos-1) % self.length
            if self.terminals[test_pos] == False:
                pos = test_pos
        return np.stack(im_deque, axis=-1)
    
    def minibatch(self, size):
        #return np.random.choice(self.data[:self.size], size=sz, replace=False)
        indices = np.random.choice(self.size, size=size, replace=False)
        x = np.zeros((size,)+self.screen_shape+(4,))
        y = np.zeros((size,)+self.screen_shape+(4,))
        for i in range(size):
            x[i] = self.stacked_frames_x(indices[i])
            y[i] = self.stacked_frames_y(indices[i])
        return x, self.actions[indices], self.rewards[indices], y, self.terminals[indices]


mini_batch_size = 32
replay_memory_size = 100000
replay_memory = MemoryBuffer(replay_memory_size, (67,104), (1,))



In [24]:
#---------------------------------Training algorithm-------------------------------
game = FlappyBird(graphics="fixed") # use "fancy" for full background, random bird color and random pipe color, use "fixed" (default) for black background and constant bird and pipe colors.
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=True, display_screen=False)
# Note: if you want to see you agent act in real time, set force_fps to False. But don't use this setting for learning, just for display purposes.

evaluation_period = 5000
total_steps = 300000
gamma = 0.99

nb_epochs = total_steps // evaluation_period
epoch=-1
scoreQ = np.zeros((nb_epochs))

p.init()
reward = 0.0
cumulated = np.zeros((total_steps))
i = 0

p.reset_game()
screen_x = process_screen(p.getScreenRGB())
stacked_x = deque([screen_x, screen_x, screen_x, screen_x], maxlen=4)
x = np.stack(stacked_x, axis=-1)





for step in range(total_steps):

    # action selection
    a_predict = greedy_action(dqn, x)
    if np.random.rand() < 0.1:
        if np.random.randint(0,10) < 2:
            a = 1
        else:
            a = 0
    else:
        a = a_predict
    action = int_to_action(a)
    
    # step
    reward_raw = p.act(action)
    reward = clip_reward_2(reward_raw, p.game_over())
    screen_y = process_screen(p.getScreenRGB())
    replay_memory.append(screen_x, a, reward, screen_y, p.game_over())
    
    
    # train
    if step>mini_batch_size:
        X,A,R,Y,D = replay_memory.minibatch(mini_batch_size)
        QY = dqn.predict(Y)
        QYmax = QY.max(1).reshape((mini_batch_size,1))
        update = R + gamma * (1-D) * QYmax
        QX = dqn.predict(X)
        QX[np.arange(mini_batch_size), A.ravel()] = update.ravel()
        dqn.train_on_batch(x=X, y=QX)
        
    #next transition 
    if(p.game_over()):
        # restart episode
        p.reset_game()
        screen_x = process_screen(p.getScreenRGB())
        stacked_x = deque([screen_x, screen_x, screen_x, screen_x], maxlen=4)
        x = np.stack(stacked_x, axis=-1)
    else: 
        # keep going
        screen_x = screen_y
        stacked_x.append(screen_x)
        x = np.stack(stacked_x, axis=-1)
        
    print("step:{}".format(step), end='\r')
    
dqn.save("temp/flappy-bucket-2/300000.h5")
print("saved")

E:\logiciels\Anaconda\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


SystemExit: 

E:\logiciels\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
#---------------------------------Evaluation algorithm-------------------------------
game = FlappyBird(graphics="fixed") # use "fancy" for full background, random bird color and random pipe color, use "fixed" (default) for black background and constant bird and pipe colors.
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=False, display_screen=True)
# Note: if you want to see you agent act in real time, set force_fps to False. But don't use this setting for learning, just for display purposes.
trials = 100
scores = np.zeros((trials))

for i in range(trials):
    p.reset_game()
    screen_x = process_screen(p.getScreenRGB())
    state = game.getGameState()
    stacked_x = deque([screen_x, screen_x, screen_x, screen_x], maxlen=4)
    x = np.stack(stacked_x, axis=-1)
    while p.game_over()!=True:            
        #action selection
        action = int_to_action(greedy_action(dqn, x))

        reward_raw = p.act(action)       
        screen_y = process_screen(p.getScreenRGB())
        scores[i] = scores[i] + reward_raw
        
        #next state
        screen_x = screen_y
        stacked_x.append(screen_x)
        x = np.stack(stacked_x, axis=-1)
    print(scores[i])
        
print(np.mean(scores))

E:\logiciels\Anaconda\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


SystemExit: 

E:\logiciels\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [28]:
#--------------------------Save dqn--------------------------
try:
    confirmation = int(input("confirm network save ?")) 
except:
    print("failed")
    confirmation = 0
if(confirmation == 1):
    name = str(input("file name ?")) 
    dqn.save("temp/flappy-bucket-2/" + name + '.h5')
else:
    print("aborted")

confirm network save ?1
file name ?nickel600000
